# Analysis of output files
## Prepare environment, functions etc.

In [1]:
import os
import sys

# Add local src directory to the path. Then we are able to import our files.
module_path = os.path.abspath(os.path.join('src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas
import matplotlib
import numpy
from helpers import get_analysis_files, getFiles, FilePair
from myDataClasses import AnalysisFile

path = 'analysisOutput/results'

In [3]:
#pandas.set_option('display.max_rows', None)
#pandas.read_csv?

In [4]:
# Important functions

def remove_bag_results(table):
    delim_index = list(table.iloc[0]).index("|")
    
    return table.iloc[:, 0:delim_index]

def get_cols_list(path: str):
    cols = pandas.read_csv(path, index_col=None, delimiter=" ", header=None)
    cols = remove_bag_results(cols)
    return list(cols.iloc[0])

def load_analysis_files(folder_path: str, column_list):
    files: AnalysisFile = get_analysis_files(folder_path)
    output_table = None
    
    for file in files:
        curr_table = pandas.read_csv(file.full_path, index_col=None, delimiter=" ", header=None)
        curr_table = remove_bag_results(curr_table)
        curr_table.columns = column_list
        curr_table["dataset"] = file.dataset
        curr_table["strategy"] = file.strategy
        
        if output_table is not None:
            output_table = output_table.append(curr_table, ignore_index=True)
        else:
            output_table = curr_table
    
    output_table = output_table.set_index(['strategy', 'dataset', 'id', "item_count"])
    return output_table

def construct_table_from(filePair: FilePair):
    solution_table = pandas.read_csv(filePair.solutionFile, header=None, index_col=None, delimiter=" ")
    data_table = pandas.read_csv(filePair.dataFile, header=None, index_col=None, delimiter=" ")
    
    item_count = data_table.iloc[0, 1]
    
    solution_table = solution_table.drop_duplicates(subset=[0], keep='first').reset_index()

    data_table = data_table.iloc[:, 4:]
    data_table = data_table[data_table.columns[::2]]

    info_table = pandas.concat([solution_table.iloc[:, 1], solution_table.iloc[:, 3], data_table.max(axis=1)], axis=1)
    info_table.columns = ["id", "best_value", "max_cost"]
    info_table["item_count"] = item_count
    return info_table

def get_info_from_datafiles(path: str):
    dataset = path.split("/")[-1]
    output_table = None
    for filePair in getFiles(path):
        curr_table = construct_table_from(filePair)
        curr_table["dataset"] = dataset
        curr_table = curr_table.set_index(["dataset", "item_count", "id"])
        
        if output_table is not None:
            output_table = output_table.append(curr_table)
        else:
            output_table = curr_table
    
    return output_table

## Put data from all analysis files into tables

In [5]:
# Create column lists

dp_cols = get_cols_list(f'{path}/DP/column_description_DP.dat')
dpweight_cols = get_cols_list(f'{path}/DPWeight/column_description_DPWeight.dat')
greedy_cols = get_cols_list(f'{path}/Greedy/column_description_Greedy.dat')
greedyone_cols = get_cols_list(f'{path}/GreedyOne/column_description_GreedyOne.dat')
fptas_cols = get_cols_list(f'{path}/FPTAS/column_description_FPTAS.dat')

In [6]:
# Load tables of all strategies
dp_table = load_analysis_files(f'{path}/DP', dp_cols)
dpweight_table = load_analysis_files(f'{path}/DPWeight', dpweight_cols)
greedy_table = load_analysis_files(f'{path}/Greedy', greedy_cols)
greedyone_table = load_analysis_files(f'{path}/GreedyOne', greedyone_cols)
fptas_table = load_analysis_files(f'{path}/FPTAS', fptas_cols)

## Get average time values for all strategies

In [7]:
# Add all table rows into 1 table
avg_times = fptas_table.drop(["relative_error"], axis=1).append(dp_table).append(dpweight_table).append(greedy_table).append(greedyone_table)

# Create a table of average times according to strategy and item_count columns
avg_times = avg_times.groupby(["strategy", "item_count"])["time"].mean().reset_index().set_index(["strategy", "item_count"])
avg_times = avg_times.round(4)

# Move all values of strategy column into separate columns
avg_times = avg_times.unstack("strategy")
avg_times.columns = avg_times.columns.droplevel()

# Save the dataframe to csv
avg_times.to_csv ('analysisOutput/avg_times.csv', header=True)

avg_times

strategy,DP,DPWeight,FPTAS,Greedy,GreedyOne
item_count,,,,,
4,93.6982,1.4304,0.8519,0.0105,0.0079
10,456.8244,7.7131,10.2270,0.0160,0.0099
15,1106.8275,21.3662,32.6015,0.0204,0.0119
20,1950.3325,37.8990,83.6230,0.0269,0.0133
22,1876.8124,51.3970,116.6782,0.0317,0.0171
25,3221.8843,61.1677,175.6165,0.0305,0.0163
27,2785.1095,72.0347,222.2353,0.0372,0.0168
30,3349.9526,85.1810,292.5677,0.0370,0.0186
32,4461.4599,103.9543,331.5438,0.0387,0.0179


## FPTAS error/item_count analysis

In [8]:
# Create info_table which contains best values and highest cost for all instances
nk_info = get_info_from_datafiles("data/NK")
zkc_info = get_info_from_datafiles("data/ZKC")
zkw_info = get_info_from_datafiles("data/ZKW")

info_table = nk_info.append(zkc_info).append(zkw_info)

In [9]:
# Prepare the table for fptas analysis which has data from info_table
fptas_analysis_table = fptas_table.reset_index().drop(columns="strategy")
fptas_analysis_table = fptas_analysis_table.set_index(["dataset", "item_count", "relative_error", "id"])
fptas_analysis_table = fptas_analysis_table.join(info_table)
fptas_analysis_table.rename(columns={'maximum_sum':'best_found_value'}, inplace=True)

# Rearrange columns
fptas_analysis_table = fptas_analysis_table[["max_cost", "best_value", "best_found_value", "time"]]

In [10]:
fptas_analysis_table["ommited_bits"] = (fptas_analysis_table.index.get_level_values("relative_error") * fptas_analysis_table["max_cost"])/fptas_analysis_table.index.get_level_values("item_count")
fptas_analysis_table["ommited_bits"] = numpy.log2(fptas_analysis_table["ommited_bits"])

fptas_analysis_table["measured_relative_error"] = (fptas_analysis_table.index.get_level_values("item_count") * numpy.power(2, numpy.floor(fptas_analysis_table["ommited_bits"])))/fptas_analysis_table["max_cost"]

In [11]:
pandas.set_option('display.max_rows', None)

fptas_analysis_table.query("item_count == 4 and relative_error == 0.1")

max_cost  best_value  \
dataset item_count id   relative_error                         
NK      4          1    0.1                 2084        1129   
                   2    0.1                 1503        2966   
                   3    0.1                 2253           0   
                   4    0.1                 1826          70   
                   5    0.1                 1755        2758   
                   6    0.1                 1725        1641   
                   7    0.1                 1676        4607   
                   8    0.1                 2341        4422   
                   9    0.1                 1635        3451   
                   10   0.1                 2388         744   
                   11   0.1                 2155        4774   
                   12   0.1                 1751        1538   
                   13   0.1                 2405        5798   
                   14   0.1                 1858        3972   
                   15   0.1                 2167        4117   
                   16   0.1                 2261        2698   
                   17   0.1                 2045           0   
                   18   0.1                 2453        2453   
                   19   0.1                 2319        3999   
                   20   0.1                 2414        4919   
                   21   0.1                 2098        2098   
                   22   0.1                 1931        1382   
                   23   0.1                 1885        4485   
                   24   0.1                 2260        3611   
                   25   0.1                 1585        3102   
                   26   0.1                 1593        2551   
                   27   0.1                 1990        3076   
                   28   0.1                 1649        1649   
                   29   0.1                 1643        2937   
                   30   0.1                 2450        5798   
                   31   0.1                 2123        4506   
                   32   0.1                 2474        5893   
                   33   0.1                 2480        2277   
                   34   0.1                 2359        2689   
                   35   0.1                 2356        2356   
                   36   0.1                 1751        1209   
                   37   0.1                 1347           0   
                   38   0.1                 2017        3414   
                   39   0.1                 2453        4899   
                   40   0.1                 2001        2310   
                   41   0.1                 2111        2024   
                   42   0.1                 2033        2625   
                   43   0.1                 2367        3331   
                   44   0.1                 2015        3271   
                   45   0.1                 1971        4966   
                   46   0.1                 1107        1792   
                   47   0.1                 2403        4753   
                   48   0.1                 2320        3966   
                   49   0.1                 2483           0   
                   50   0.1                 2362        2717   
                   51   0.1                 2295           0   
                   52   0.1                 2438        3895   
                   53   0.1                 2244        4212   
                   54   0.1                  890         523   
                   55   0.1                 2137        4588   
                   56   0.1                 2097        2986   
                   57   0.1                 1933        3438   
                   58   0.1                 2187        2090   
                   59   0.1                 2431        4343   
                   60   0.1                 2260        4469   
                   61   0.1                 2498           0   
        

In [39]:
error_group = fptas_analysis_table.groupby(["item_count", "relative_error"])["measured_relative_error"]
time_group = fptas_analysis_table.groupby(["item_count", "relative_error"])["time"]

error_max = error_group.max().reset_index().set_index(["item_count", "relative_error"]).rename(columns={'measured_relative_error':'measured_relative_error_max'})
error_avg = error_group.mean().reset_index().set_index(["item_count", "relative_error"]).rename(columns={'measured_relative_error':'measured_relative_error_avg'})
time_sum = time_group.sum().reset_index().set_index(["item_count", "relative_error"])

time_sum["time"] = time_sum["time"]/1000

error_max.join(error_avg).join(time_sum)

measured_relative_error_max  \
item_count relative_error                                
4          0.1                                0.100000   
           0.2                                0.200000   
           0.3                                0.299941   
           0.4                                0.400000   
           0.5                                0.500000   
10         0.1                                0.099984   
           0.2                                0.199969   
           0.3                                0.299766   
           0.4                                0.399938   
           0.5                                0.500000   
15         0.1                                0.100000   
           0.2                                0.200000   
           0.3                                0.297905   
           0.4                                0.400000   
           0.5                                0.500000   
20         0.1                                0.099860   
           0.2                                0.199719   
           0.3                                0.299065   
           0.4                                0.399438   
           0.5                                0.499439   
22         0.1                                0.099491   
           0.2                                0.198982   
           0.3                                0.299957   
           0.4                                0.397965   
           0.5                                0.499114   
25         0.1                                0.098912   
           0.2                                0.197824   
           0.3                                0.292344   
           0.4                                0.395648   
           0.5                                0.481275   
27         0.1                                0.100000   
           0.2                                0.200000   
           0.3                                0.299584   
           0.4                                0.400000   
           0.5                                0.490213   
30         0.1                                0.100000   
           0.2                                0.200000   
           0.3                                0.295749   
           0.4                                0.400000   
           0.5                                0.495740   
32         0.1                                0.099283   
           0.2                                0.198565   
           0.3                                0.299196   
           0.4                                0.397130   
           0.5                                0.494447   
35         0.1                                0.099397   
           0.2                                0.198793   
           0.3                                0.296140   
           0.4                                0.397586   
           0.5                                0.498998   
37         0.1                                0.099412   
           0.2                                0.198825   
           0.3                                0.294308   
           0.4                                0.397649   
           0.5                                0.500000   
40         0.1                                0.099283   
           0.2                                0.198565   
           0.3                                0.297122   
           0.4                                0.397130   
           0.5                                0.484115   

                           measured_relative_error_avg         time  
item_count relative_error                                            
4          0.1                                0.067010     2.977725  
           0.2                                0.134020     2.030097  
           0.3                                0.222117     1.405725  
           0.4                                0.268040     1.090785  
          

In [33]:
fptas_analysis_table["test"] = numpy.abs(fptas_analysis_table["best_value"] - fptas_analysis_table["best_found_value"])/fptas_analysis_table["best_value"]
fptas_analysis_table.fillna(0.0, inplace=True)
one = fptas_analysis_table.query('test != 1.0 and test != 0.0').shape[0]
two = fptas_analysis_table.query('test == 0.0').shape[0]
three = fptas_analysis_table.query('test == 1.0').shape[0]
four = fptas_analysis_table.query('relative_error < test and test != 1').shape[0]

print("Četnosti naměřených relativních chyb:\n")
print(f'(e == 0): {two}\n(e == 1): {three}\n(0 < e < 1): {one}')
print(f'(e_wanted < e_measured and e_measured != 1): {four}')


Četnosti naměřených relativních chyb:

(e == 0): 47820
(e == 1): 21
(0 < e < 1): 21339
(e_wanted < e_measured and e_measured != 1): 0
